In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM, concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
df= pd.read_excel('Classification_balanced_Categories.xlsx')# Data used in Classification 
data = df

# Data splitting, preprocessing 

In [ ]:
# Get df values
X = data.values
X[X==0] = 1e-5



X_train_base, X_test_base = train_test_split(X, test_size=0.2, random_state=100, shuffle=True)
X_train_base, X_val_base = train_test_split(X_train_base, test_size=0.05, random_state=50, shuffle=False)

# Normalization
scaler = StandardScaler()

# Create and normalize train datasets for 1D
list = [5] # Labels index
num_time_steps = 100
norm_log_t_train = scaler.fit_transform(np.log(X_train_base[:, 5:105].astype(float)))
norm_log_p_train = scaler.fit_transform(np.log(X_train_base[:, 105:205].astype(float)))
norm_log_dp_train = scaler.fit_transform(np.log(X_train_base[:, 205:305].astype(float)))
norm_target_train = (X_train_base[:, list].astype(int))

# Discard time, normalize logP, logdP for 2D
X_train = np.stack([norm_log_p_train,   norm_log_dp_train], axis=2)  # Stacking 'dp' and 'derp'
X_train_reshaped = X_train.reshape(-1, num_time_steps, 2)
train_label = norm_target_train -1

X_train_1 = np.hstack([ norm_log_t_train, norm_log_dp_train])#norm_log_p_train, norm_log_t_train,
train_label = norm_target_train -1

# Create and normalize test datasets 1D
norm_t_test = scaler.fit_transform(np.log(X_test_base[:, 5:105].astype(float)))
norm_log_p_test = scaler.fit_transform(np.log(X_test_base[:, 105:205].astype(float)))
norm_log_dp_test = scaler.fit_transform(np.log(X_test_base[:, 205:305].astype(float)))
norm_target_test = (X_test_base[:, list].astype(int))

# Discard time, normalize logP, logdP 2D
X_test = np.stack([norm_log_p_test,  norm_log_dp_test], axis=2)  # Stacking 'dp' and 'derp'
X_test_reshaped = X_test.reshape(-1, num_time_steps, 2)
test_label = norm_target_test 

X_test_1 = np.hstack([ norm_t_test, norm_log_dp_test])#norm_log_p_test, norm_t_test,
test_label = norm_target_test -1

# Create and normalize Val datasets
norm_t_val = scaler.fit_transform(np.log(X_val_base[:, 5:105].astype(float)))
norm_log_p_val = scaler.fit_transform(np.log(X_val_base[:, 105:205].astype(float)))
norm_log_dp_val = scaler.fit_transform(np.log(X_val_base[:, 205:305].astype(float)))
norm_target_val = (X_val_base[:, list].astype(int))

# Discard time, normalize logP, logdP
X_val = np.stack([norm_log_p_val,  norm_log_dp_val], axis=2)  # Stacking 'dp' and 'derp'
X_val_reshaped = X_val.reshape(-1, num_time_steps, 2)
val_label = norm_target_val

X_val_1 = np.hstack([norm_t_val,  norm_log_dp_val])#norm_log_p_val,
val_label = norm_target_val -1


# 1D
train_sets1 = [X_train_1, train_label] # can add the lebel to the array ",train_label"
test_sets1  = [X_test_1, test_label]
val_sets1  = [X_val_1, val_label]

# 2D
train_sets2 = [X_train, test_label] # can add the lebel to the array ",train_label"
test_sets2  = [X_test, train_label]
val_sets2  = [X_val, val_label]

# 1 num_conv_layers 

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, BatchNormalization, Flatten, Activation

def create_model(num_conv_layers):
    model = Sequential()
    # Input layer
    model.add(Conv1D(filters=128, kernel_size=7, activation='relu', input_shape=(200, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # Additional Conv layers based on num_conv_layers parameter
    for _ in range(1, num_conv_layers):
        model.add(Conv1D(filters=128, kernel_size=3 if _ % 2 == 0 else 5, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2))
    
    # Flatten and Dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Creating models with different numbers of convolutional layers
models = {}
for i in range(1, 4):  # Assuming the original model had 3 Conv1D layers
    models[f'model_with_{i}_conv_layers'] = create_model(i)

# Now, you can train and evaluate each model separately

In [ ]:
history_dict = {}
for key, model in models.items():
    print(f"Training {key}...")
    history = model.fit(train_sets1[0], train_sets1[1], epochs=50, validation_data=(test_sets1[0], test_sets1[1]), batch_size=20)
    history_dict[key] = history
    model.evaluate(test_sets1[0], test_sets1[1])


In [1]:
# Plot settings
plt.figure(figsize=(12, 8))
colors = ['blue', 'green', 'red']  # Different colors for each model
labels = ['1 Conv Layer', '2 Conv Layers', '3 Conv Layers']  # Labels for each line

# Loop through the history of each model and plot
for (key, history), color, label in zip(history_dict.items(), colors, labels):
    epochs = range(1, 51)  # 10 epochs, adjust if necessary
    training_loss = history.history['loss']
    validation_loss = history.history['val_loss']

    # Plot training and validation loss
    #plt.plot(epochs, training_loss, 'o-', color=color, label=f'Training Loss - {label}')
    plt.plot(epochs, validation_loss, 's--', color=color, label=f'Validation Loss - {label}')

# Adding titles and labels
plt.title('Validation Loss by Convolutional Layer Count Over Epochs')
plt.xlabel('Epochs',fontsize=14)
plt.ylabel('Loss',fontsize=14)
plt.legend()

# Adding grid with major and minor lines
plt.grid(which='major', linestyle='-', linewidth='0.5', color='gray')  # Major grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')  # Minor grid
plt.minorticks_on()  # Enable minor ticks

# Save the figure at a high resolution
plt.savefig('Conv_layer_variation_effect_classification.png', dpi=1200)

# Show plot
plt.show()

NameError: name 'plt' is not defined

# 2 Kernel_variation_effect

In [ ]:
From keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, BatchNormalization, Flatten

def create_model(kernel_sizes):
    model = Sequential()
    # Input layer
    model.add(Conv1D(filters=128, kernel_size=kernel_sizes[0], activation='relu', input_shape=(200, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # Additional Conv layers based on kernel_sizes list
    for size in kernel_sizes[1:]:
        model.add(Conv1D(filters=128, kernel_size=size, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2))
    
    # Flatten and Dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define different sets of kernel sizes for the experiment
kernel_configurations = [
    [7, 5, 3],  # Original sizes
    [3, 3, 3],  # Smaller kernels
    [9, 7, 5]   # Larger kernels
]

# Creating models with different kernel sizes
models = {}
for i, sizes in enumerate(kernel_configurations, start=1):
    model_key = f'model_with_kernel_sizes_{sizes}'
    models[model_key] = create_model(sizes)

In [ ]:
# Train and evaluate each model
history_dict = {}
for key, model in models.items():
    print(f"Training {key}...")
    history = model.fit(train_sets1[0], train_sets1[1], epochs=50, validation_data=(test_sets1[0], test_sets1[1]), batch_size=20)
    history_dict[key] = history
    evaluation = model.evaluate(test_sets1[0], test_sets1[1])
    print(f"Evaluation {key}: Loss = {evaluation[0]}, Accuracy = {evaluation[1]}")

In [ ]:
# Plot settings
plt.figure(figsize=(12, 8))
colors = ['blue', 'green', 'red']  # Different colors for each model configuration
labels = ['Kernel Sizes: [7, 5, 3]', 'Kernel Sizes: [3, 3, 3]', 'Kernel Sizes: [9, 7, 5]']  # Labels based on kernel sizes

# Assuming history_dict is correctly updated to include models keyed by their kernel configuration
# Loop through the history of each model and plot
for (key, history), color, label in zip(history_dict.items(), colors, labels):
    epochs = range(1, 51)  # Adjust the range based on the actual number of epochs trained
    validation_loss = history.history['val_loss']

    # Plot validation loss
    plt.plot(epochs, validation_loss, 's--', color=color, label=f'Validation Loss - {label}')

# Adding titles and labels
plt.title('Validation Loss for Different Kernel Configurations Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend()

# Adding grid with major and minor lines
plt.grid(which='major', linestyle='-', linewidth='0.5', color='gray')  # Major grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')  # Minor grid
plt.minorticks_on()  # Enable minor ticks

# Save the figure at a high resolution
plt.savefig('Kernel_variation_effect_classification.png', dpi=1200)

# Show plot
plt.show()

# 3 Pooling configurations

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, AveragePooling1D, Dense, BatchNormalization, Flatten

def create_model(kernel_sizes, pool_size, pooling_type='average'):
    model = Sequential()
    # Input layer
    model.add(Conv1D(filters=128, kernel_size=kernel_sizes[0], activation='relu', input_shape=(200, 1)))
    model.add(BatchNormalization())
    
    if pooling_type == 'average':
        model.add(AveragePooling1D(pool_size=pool_size))
    else:
        model.add(MaxPooling1D(pool_size=pool_size))
    
    # Additional Conv layers based on kernel_sizes list
    for size in kernel_sizes[1:]:
        model.add(Conv1D(filters=128, kernel_size=size, activation='relu'))
        model.add(BatchNormalization())
        
        if pooling_type == 'average':
            model.add(AveragePooling1D(pool_size=pool_size))
        else:
            model.add(MaxPooling1D(pool_size=pool_size))
    
    # Flatten and Dense layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Pooling configurations for the experiment
pooling_configurations = [
    {'pool_size': 2, 'pooling_type': 'average'},
    {'pool_size': 3, 'pooling_type': 'average'},
    {'pool_size': 2, 'pooling_type': 'max'}
]

# Kernel sizes used for each model
kernel_sizes = [7, 5, 3]  # Adjust as necessary based on previous configurations

# Creating models with different pooling strategies and sizes
models = {}
for config in pooling_configurations:
    model_key = f"model_pool_{config['pool_size']}_{config['pooling_type']}"
    models[model_key] = create_model(kernel_sizes, **config)

In [ ]:
# Train and evaluate each model
history_dict = {}
for key, model in models.items():
    print(f"Training {key}...")
    history = model.fit(train_sets1[0], train_sets1[1], epochs=100, validation_data=(test_sets1[0], test_sets1[1]), batch_size=20)
    history_dict[key] = history
    evaluation = model.evaluate(test_sets1[0], test_sets1[1])
    print(f"Evaluation {key}: Loss = {evaluation[0]}, Accuracy = {evaluation[1]}")

In [ ]:
# Plot settings
plt.figure(figsize=(12, 8))
colors = ['blue', 'green', 'red']  # Different colors for each pooling configuration
labels = [
    'Average Pooling Size 2', 
    'Average Pooling Size 3', 
    'Max Pooling Size 2'
]  # Labels based on pooling type and size

# Loop through the history of each model and plot
for (key, history), color, label in zip(history_dict.items(), colors, labels):
    epochs = range(1, 101)  # Adjust the range based on the actual number of epochs trained
    validation_loss = history.history['val_loss']

    # Plot validation loss
    plt.plot(epochs, validation_loss, 'o--', color=color, label=f'{label} Validation Loss')

# Adding titles and labels
plt.title('Impact of Pooling Strategies on Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend()

# Adding grid with major and minor lines
plt.grid(which='major', linestyle='-', linewidth='0.5', color='gray')  # Major grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')  # Minor grid
plt.minorticks_on()  # Enable minor ticks

# Save the figure at a high resolution
plt.savefig('Impact_of_Pooling_Strategies_classification.png', dpi=1200)

# Show plot
plt.show()

# 4 Impact of Neuron Counts in Dense Layer

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, BatchNormalization, Flatten

def create_model(kernel_sizes, dense_neurons):
    model = Sequential()
    # Input layer
    model.add(Conv1D(filters=128, kernel_size=kernel_sizes[0], activation='relu', input_shape=(200, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # Additional Conv layers based on kernel_sizes list
    for size in kernel_sizes[1:]:
        model.add(Conv1D(filters=128, kernel_size=size, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2))
    
    # Flatten the output before passing it to the Dense layers
    model.add(Flatten())

    # Adjustable Dense layer
    model.add(Dense(dense_neurons, activation='relu'))  # Adjusting the number of neurons

    # Output layer
    model.add(Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Different neuron counts in the dense layer
neuron_configurations = [64, 128, 256]  # Example configurations

# Creating models with different numbers of neurons in the dense layer
models = {}
for neurons in neuron_configurations:
    model_key = f"model_with_{neurons}_neurons"
    models[model_key] = create_model([7, 5, 3], neurons)  # Assuming kernel sizes are fixed



In [ ]:
# Train and evaluate each model
history_dict = {}
for key, model in models.items():
    print(f"Training {key}...")
    history = model.fit(train_sets1[0], train_sets1[1], epochs=100, validation_data=(test_sets1[0], test_sets1[1]), batch_size=20)
    history_dict[key] = history
    evaluation = model.evaluate(test_sets1[0], test_sets1[1])
    print(f"Evaluation {key}: Loss = {evaluation[0]}, Accuracy = {evaluation[1]}")

In [ ]:
# Plot settings
plt.figure(figsize=(12, 8))
colors = ['blue', 'green', 'red']  # Different colors for each neuron configuration
labels = ['64 Neurons', '128 Neurons', '256 Neurons']  # Labels based on neuron counts in the dense layer

# Loop through the history of each model and plot
for (key, history), color, label in zip(history_dict.items(), colors, labels):
    epochs = range(1, 101)  # Adjust the range based on the actual number of epochs trained
    validation_loss = history.history['val_loss']

    # Plot validation loss
    plt.plot(epochs, validation_loss, 'o-', color=color, label=f'{label} Validation Loss')

# Adding titles and labels
plt.title('Impact of Neuron Counts in Dense Layer on Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend()

# Adding grid with major and minor lines
plt.grid(which='major', linestyle='-', linewidth='0.5', color='gray')  # Major grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')  # Minor grid
plt.minorticks_on()  # Enable minor ticks

# Save the figure at a high resolution
plt.savefig('Impact_of_Neuron_Counts_classification.png', dpi=1200)

# Show plot
plt.show()

# 5 Impact of Activation Functions

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, BatchNormalization, Flatten

def create_model(kernel_sizes, activation_function):
    model = Sequential()
    # Input layer
    model.add(Conv1D(filters=128, kernel_size=kernel_sizes[0], activation=activation_function, input_shape=(200, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # Additional Conv layers based on kernel_sizes list
    for size in kernel_sizes[1:]:
        model.add(Conv1D(filters=128, kernel_size=size, activation=activation_function))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2))
    
    # Flatten the output before passing it to the Dense layers
    model.add(Flatten())

    # Dense layer with configurable activation function
    model.add(Dense(128, activation=activation_function))

    # Output layer
    model.add(Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Different activation functions to test
activation_functions = ['relu', 'sigmoid', 'tanh']

# Creating models with different activation functions
models = {}
for activation in activation_functions:
    model_key = f"model_with_{activation}_activation"
    models[model_key] = create_model([7, 5, 3], activation)  # Assuming kernel sizes are fixed


In [ ]:
# Train and evaluate each model
history_dict = {}
for key, model in models.items():
    print(f"Training {key}...")
    history = model.fit(train_sets1[0], train_sets1[1], epochs=100, validation_data=(test_sets1[0], test_sets1[1]), batch_size=20)
    history_dict[key] = history
    evaluation = model.evaluate(test_sets1[0], test_sets1[1])
    print(f"Evaluation {key}: Loss = {evaluation[0]}, Accuracy = {evaluation[1]}")

In [ ]:
# Plot settings
plt.figure(figsize=(12, 8))
colors = ['blue', 'green', 'red']  # Different colors for each activation function
labels = ['ReLU Activation', 'Sigmoid Activation', 'Tanh Activation']  # Labels based on activation functions

# Loop through the history of each model and plot
for (key, history), color, label in zip(history_dict.items(), colors, labels):
    epochs = range(1, 101)  # Adjust the range based on the actual number of epochs trained
    validation_loss = history.history['val_loss']

    # Plot validation loss
    plt.plot(epochs, validation_loss, 'o-', color=color, label=f'{label} Validation Loss')

# Adding titles and labels
plt.title('Impact of Activation Functions on Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend()

# Adding grid with major and minor lines
plt.grid(which='major', linestyle='-', linewidth='0.5', color='gray')  # Major grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')  # Minor grid
plt.minorticks_on()  # Enable minor ticks

# Save the figure at a high resolution
plt.savefig('Impact_of_ Activation_Functions_classification.png', dpi=1200)

# Show plot
plt.show()

# 6 Batch Normalization Influence

In [ ]:
def create_model(kernel_sizes, use_batch_norm=True):
    model = Sequential()
    # Input layer
    model.add(Conv1D(filters=128, kernel_size=kernel_sizes[0], activation='relu', input_shape=(200, 1)))
    if use_batch_norm:
        model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    
    # Additional Conv layers based on kernel_sizes list
    for size in kernel_sizes[1:]:
        model.add(Conv1D(filters=128, kernel_size=size, activation='relu'))
        if use_batch_norm:
            model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2))
    
    # Flatten the output before passing it to the Dense layers
    model.add(Flatten())

    # Dense layer
    model.add(Dense(128, activation='relu'))

    # Output layer
    model.add(Dense(4, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Batch normalization configurations for the experiment
batch_norm_configurations = [True, False]

# Kernel sizes used for each model
kernel_sizes = [3, 3, 3]  # Adjust as necessary based on previous configurations

# Creating models with and without batch normalization
models = {}
for use_batch_norm in batch_norm_configurations:
    model_key = f"model_with_batch_norm_{use_batch_norm}"
    models[model_key] = create_model(kernel_sizes, use_batch_norm)


In [ ]:
# Train and evaluate each model
history_dict = {}
for key, model in models.items():
    print(f"Training {key}...")
    history = model.fit(train_sets1[0], train_sets1[1], epochs=100, validation_data=(test_sets1[0], test_sets1[1]), batch_size=20)
    history_dict[key] = history
    evaluation = model.evaluate(test_sets1[0], test_sets1[1])
    print(f"Evaluation {key}: Loss = {evaluation[0]}, Accuracy = {evaluation[1]}")

In [ ]:
# Plot settings
plt.figure(figsize=(12, 8))
colors = ['blue', 'green']  # Different colors for with and without Batch Normalization
labels = ['With Batch Normalization', 'Without Batch Normalization']  # Labels based on Batch Normalization usage

# Loop through the history of each model and plot
for (key, history), color, label in zip(history_dict.items(), colors, labels):
    epochs = range(1, 101)  # Adjust the range based on the actual number of epochs trained
    validation_loss = history.history['val_loss']

    # Plot validation loss
    plt.plot(epochs, validation_loss, 'o-', color=color, label=f'{label} Validation Loss')

# Adding titles and labels
plt.title('Impact of Batch Normalization on Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend()

# Adding grid with major and minor lines
plt.grid(which='major', linestyle='-', linewidth='0.5', color='gray')  # Major grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')  # Minor grid
plt.minorticks_on()  # Enable minor ticks

# Save the figure at a high resolution
plt.savefig('Impact_of_ Batch_Normalization_classification.png', dpi=1200)


# Show plot
plt.show()

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

# Sample data: time and pressure
time_pressure = np.array([0, 1, 2, 3, 4, 5])  # Common time points for the pressure
pressure = np.array([10, 11, 9, 13, 12, 14])  # Pressure readings

# Sample data: time and derivative
time_derivative = np.array([0.5, 1.5, 2.5, 3.5, 4.5])  # Different time points
derivative = np.array([0.1, -0.2, 0.15, -0.05, 0.1])  # Derivative readings

# Interpolation function for the derivative
interpolation_function = interp1d(time_derivative, derivative, kind='linear', fill_value="extrapolate")

# Interpolated derivative values at the pressure time points
interpolated_derivative = interpolation_function(time_pressure)

print("Interpolated Derivative Values:", interpolated_derivative)